# NFT Authenticity and Prediction using Sentiment Analysis and Deep Learning

## Imports

In [12]:
import requests,json
import random
import asyncio
import time
import tqdm
import tqdm.asyncio
import nest_asyncio
nest_asyncio.apply()
from pprint import pprint
from aiohttp import ClientSession, TCPConnector

## Data Collection

Sample Data

In [9]:
sample_addresses = ['evolved-apes-inc','oraclenft','boredapeyachtclub','neo-tokyo-identities','cool-cats-nft','cryptopunks','veefriends']

### OpenSea Metrics from Contract Addresses

Asynchronous Data Collection

In [38]:
async def fetchData(url,session):
    async with session.get(url) as response:
        return await response.read()

async def run():
    tasks = []
    connector = TCPConnector()
    url = "https://api.opensea.io/api/v1/collection/{}"
    async with ClientSession(connector=connector) as session:
        for i in range(len(sample_addresses)):
            task = asyncio.ensure_future(fetchData(url.format(sample_addresses[i]),session))
            tasks.append(task)
        responses = []
        for f in tqdm.tqdm(asyncio.as_completed(tasks), total=len(tasks)):
            responses.append(await f)
        for response in responses:
            response = json.loads(response.decode('utf8'))
            name = response['collection']['name']
            floor = response['collection']['stats']['floor_price']
            volume = response['collection']['stats']['total_volume']
            pprint(f'{name}: Floor: {floor}ETH --- Total Volume: {volume}ETH')


Run Script

In [39]:
loop = asyncio.get_event_loop()
future = asyncio.ensure_future(run())
loop.run_until_complete(future)

100%|██████████| 7/7 [00:00<00:00, 19.23it/s]

'Cool Cats NFT: Floor: 7.0ETH --- Total Volume: 58545.89960225042ETH'
'VeeFriends: Floor: 8.35ETH --- Total Volume: 31773.048003191412ETH'
'Neo Tokyo Identities: Floor: 16.8ETH --- Total Volume: 10899.947364958138ETH'
'Bored Ape Yacht Club: Floor: 49.269ETH --- Total Volume: 245005.89893046283ETH'
'Evolved Apes Inc: Floor: 0.0011ETH --- Total Volume: 1155.887264119253ETH'
'Oracle (official): Floor: 0.0ETH --- Total Volume: 27.592120000000005ETH'
'CryptoPunks: Floor: NoneETH --- Total Volume: 728905.0712493036ETH'
